In [ ]:
from analysis import load_data_and_stim, run_analysis
import numpy as np
import cortex
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel

In [ ]:
data_path = '../data/'
results_path = '../res/'

surfaces = dict(S1='food_S1',
                S2='food_S2',
                S3='food_S3',
                S4='food_S4')

xfms = dict(S1='auto_r4',
            S2='auto',
            S3='auto',
            S4='auto')

data_names = dict(S1='{}/S1/r{}_{}.nii'.format(data_path,'{}','{}'),
                  S2='{}/S2/r{}_{}.nii'.format(data_path,'{}','{}'),
                  S3='{}/S3/r{}_{}.nii'.format(data_path,'{}','{}'),
                  S4='{}/S4/r{}_{}.nii'.format(data_path,'{}','{}')
                 )

stim_paths = dict(S1='{}S1/stimuli/'.format(data_path),
                  S2='{}S2/stimuli/'.format(data_path),
                  S3='{}S3/stimuli/'.format(data_path),
                  S4='{}S4/stimuli/'.format(data_path)
                 )
                 
data_runs = dict(S1 = dict(BW = [1,2,3,4]),
                 S2 = dict(BW = [1,2,3,4,5,6,7,8,9]),
                 S3 = dict(BW = [1,2,3,4,5,6,7,8,9]),
                 S4 = dict(BW = [1,2,3,4,5,6,7,8,9]))

run_maps = dict(S1 = dict(BW = [1,2,3,1]),
                 S2 = dict(BW = [1,2,3,4,5,6,7,8,9]),
                 S3 = dict(BW = [1,2,3,4,5,6,7,8,9]),
                 S4 = dict(BW = [1,2,3,4,5,6,7,8,9]))

cond_map = dict() # constant for all for now, can be changed for other contrasts

cond_map['BW'] = dict(baseline = 'baseline',
                     food = 'food',
                     adult = 'face',
                     body = 'body',
                     house = 'house',
                     words = 'words')

In [ ]:
cname = 'BW'
all_subjects = ['S1','S2','S3','S4']
SMOOTHING = 0.5

do_run = True # change to False to load previously saved results
 
if do_run:
    
    vols = dict()

    for subject in all_subjects:
        print(subject)
        data_etc =  load_data_and_stim(surfaces[subject], xfms[subject], stim_paths[subject],
                                        data_names[subject], data_runs[subject], run_maps[subject], 
                                        cond_map, cname, SMOOTHING = SMOOTHING)
        cond_names = data_etc[3]
        contrasts = dict()
        for cond in cond_names:
            if cond != 'baseline':
                c = np.array([-1/(len(cond_names)-2) for i in range(len(cond_names))])
                c[cond_names.index(cond)] = 1
                c[cond_names.index('baseline')] = 0
                contrasts['{}'.format(cond)] = c

        # run contrasts
        vols[subject], params = run_analysis(contrasts, data_etc[0], data_etc[1], data_etc[2], 
                                    cond_names,data_etc[4], load_NSD=False, mask_MNI=0.5)

        # save
        res_name = '{}res_{}_{}_{}.npy'.format(results_path,subject,cname,SMOOTHING)
        np.save(res_name,vols[subject])
else:
    vols = dict()
    for subject in all_subjects:
        res_name = '{}res_{}_{}_{}.npy'.format(results_path,subject,'BW',0.5)
        vols[subject] = np.load(res_name,allow_pickle=True)[()]
    
    

In [ ]:
vols_to_show = dict()


for subject in all_subjects:
    mask = cortex.db.get_mask(surfaces[subject], xfms[subject],'thick')
    vols_to_show['{}_tval_food'.format(subject)] = cortex.Volume(vols[subject]['tval_food'],surfaces[subject], 
                                                                 xfms[subject], vmin = 0, vmax=10, 
                                                                 cmap='viridis')
    vols_to_show['{}_pval_food'.format(subject)] = cortex.Volume(vols[subject]['pval_food'],surfaces[subject], 
                                                                 xfms[subject], vmin = 0, vmax=1, 
                                                                 cmap='Blues')

In [ ]:
for s,v in vols_to_show.items():
    cortex.quickflat.make_png('{}{}_flatmap.png'.format(results_path, s),
                              v,with_colorbar=False,with_curvature=True,labelsize=40,linewidth=2,
                             height=1024);


In [ ]:
from save_3d_views import save_3d_views

for s, v in vols_to_show.items():
    if  s[3]=='t':
        save_3d_views(v,results_path,s,
                  list_views=['bottom'], list_surfaces=['inflated'],with_labels=True,size=(1024*4, 768*4),
                  radius=1000)
    elif s[3]=='p':
        save_3d_views(v,results_path,s,
                  list_views=['bottom'], list_surfaces=['halfinflated'],size=(1024*4, 768*4),
                  radius=1000,opacity=0.8,with_labels=False,overlays_visible=[])

In [ ]:
vols_to_show = dict()

tmp_food_MNI = 0
tmp_face_MNI = 0
tmp_words_MNI = 0
tmp_body_MNI = 0
tmp_house_MNI = 0
tmp_food_face_MNI = 0


for subject in all_subjects: # transformation to MNI smoothes the contrast. 0.2 is chosen as a threshold 
    tmp_food_MNI = tmp_food_MNI+(vols[subject]['mni_pval_food']>0.2)*1.0 
    tmp_face_MNI = tmp_face_MNI+(vols[subject]['mni_pval_face']>0.2)*1.0
    tmp_words_MNI = tmp_words_MNI+(vols[subject]['mni_pval_words']>0.2)*1.0
    tmp_body_MNI = tmp_body_MNI+(vols[subject]['mni_pval_body']>0.2)*1.0
    tmp_house_MNI = tmp_house_MNI+(vols[subject]['mni_pval_house']>0.2)*1.0
    
vols_to_show['MNI_sum_food'] = cortex.Volume(tmp_food_MNI,'MNI','atlas',vmin=0,vmax=4) 
vols_to_show['MNI_sum_face'] = cortex.Volume(tmp_face_MNI,'MNI','atlas',vmin=0,vmax=4) 
vols_to_show['MNI_sum_words'] = cortex.Volume(tmp_words_MNI,'MNI','atlas',vmin=0,vmax=4)
vols_to_show['MNI_sum_body'] = cortex.Volume(tmp_body_MNI,'MNI','atlas',vmin=0,vmax=4)
vols_to_show['MNI_sum_house'] = cortex.Volume(tmp_house_MNI,'MNI','atlas',vmin=0,vmax=4)

cortex.webshow(vols_to_show,overlays_visible=[])   

In [ ]:
for s, v in vols_to_show.items():
    if 'body' in s:
        save_3d_views(v,results_path,s,
                      list_views=['lateral'], list_surfaces=['halfinflated'],with_labels=False,size=(1024*4, 768*4),
                      radius=1000, overlays_visible=[],opacity=0.8)
    else:
        save_3d_views(v,'/Users/lwehbe/Desktop/flywheel/results/',s,
                      list_views=['bottom'], list_surfaces=['halfinflated'],with_labels=False,size=(1024*4, 768*4),
                      radius=1000, overlays_visible=[],opacity=0.8)

In [ ]:
vol_new = cortex.Volume(vols_to_show['MNI_sum_food'].data.copy(),'MNI','atlas',
                        vmin=1.4, vmax=4.2,cmap = 'nipy_spectral_r')

save_3d_views(vol_new,results_path,'localizer_sum_v2',
              list_views=['bottom','lateral'], list_surfaces=['halfinflated'],with_labels=False,
              size=(1024*4, 768*4),radius=1000, overlays_visible=[],opacity=1)

save_3d_views(vol_new,results_path,'localizer_sum_v2',
              list_views=['flat'], list_surfaces=['flatmap'],with_labels=False,
              size=(1024*4, 768*4),radius=1000, overlays_visible=[],opacity=1)